This function computes the (linear) interpolant for evaluating the differential equation:

\begin{equation}
\dfrac{d \phi}{ds} = -\dfrac{\cos^2(\phi)[S_x^{11}(\mathbf{x}_0)\cos(\phi)+S_y^{11}(\mathbf{x}_0)\sin(\phi)]+[S_x^{12}(\mathbf{x}_0)\cos(\phi)+S_y^{11}(\mathbf{x}_0)\sin(\phi)]\sin(2\phi)+\sin^2(\phi)[S_x^{22}(\mathbf{x}_0)\cos(\phi)+S_y^{22}(\mathbf{x}_0)\sin(\phi)]}{\sin(2\phi)[S^{22}(\mathbf{x}_0)-S^{11}(\mathbf{x}_0)]+2\cos(2\phi)S^{12}(\mathbf{x}_0)} \label{eq: phi_prime} \tag{1}
\end{equation}

at every point $ (\mathbf{x}, \phi) $.

To do so, we first need to evaluate the right-hand-side of eq. \ref{eq: phi_prime} over a rectangular meshgrid $ X, Y, \phi $ (see *__phi_prime*) before computing the interpolant of $ \dot{\phi} $ (see *_phi_prime*).

In [1]:
# Import numpy
import numpy as np

# Import math symbols
from math import cos, sin, pi

# Import numba (used for high performance scientific computing)
from numba import jit, prange

In [1]:
@jit(parallel=True)
def __phi_prime(S11, S12, S22, S11_x, S11_y, S12_x, S12_y, S22_x, S22_y):
    '''
    Computes phi_prime and phi_prime_denom (denominator of phi prime) over gridded domain
    based on the elements of the rate of strain tensor
    
    Parameters:
        Sij:             array(Ny, Nx), ij-element of the rate of strain tensor over the gridded domain 
        Sij_x:           array(Ny, Nx), derivative with respect to x-th coordinate of the ij-element of the rate of strain tensor over the gridded domain 
        Sij_y:           array(Ny, Nx), derivative with respect to y-th coordinate of the ij-element of the rate of strain tensor over the gridded domain 
    
    Returns:
        phi:             array(360, ), angles
        phi_prime:       array(Ny, Nx, 360), contains phi_prime over gridded domain and angles
        phi_prime_denom: array(Ny, Nx, 360), contains denominator of phi_prime over gridded domain and angles. This specifies the domain of existence.
    '''
    
    # angles
    phi = np.linspace(0, 2*pi, 360)
    
    # define arrays
    phi_prime = np.zeros((S11.shape[0], S11.shape[1], phi.shape[0]))
    phi_prime_denom = phi_prime.copy()
    
    # iterate over meshgrid in y-direction
    for i in prange(S11.shape[0]):
        
        # iterate over meshgrid in x-direction
        for j in prange(S11.shape[1]):
            
            # elements of rate of strain
            S11_ = S11[i, j]
            S12_ = S12[i, j]
            S22_ = S22[i, j]
            
            # spatial derivative of elements of rate of strain
            S11_x_ = S11_x[i, j]
            S11_y_ = S11_y[i, j]
            
            S12_x_ = S12_x[i, j]
            S12_y_ = S12_y[i, j]
            
            S22_x_ = S22_x[i, j]
            S22_y_ = S22_y[i, j]

            # iterate over phi
            for p in prange(phi.shape[0]):
                
                phi_ = phi[p]
                
                # [S11_x*cos(phi_)+S11_y*sin(phi)]*cos^2(phi)
                phi_prime_num = (S11_x_*cos(phi_)+S11_y_*sin(phi_))*cos(phi_)**2
                
                # [S12_x*cos(phi_)+S12_y*sin(phi)]*sin(2*phi)
                phi_prime_num += (S12_x_*cos(phi_)+S12_y_*sin(phi_))*sin(2*phi_)
                
                # [S11_x*cos(phi_)+S11_y*sin(phi)]*sin^2(phi)
                phi_prime_num += (S22_x_*cos(phi_)+S22_y_*sin(phi_))*sin(phi_)**2
    
                # sin(2*phi)*(S22-S11)+2*cos(2*phi)*S12
                phi_prime_denom[i, j, p] = sin(2*phi_)*(S22_-S11_)+2*cos(2*phi_)*S12_
            
                phi_prime[i, j, p] = -phi_prime_num/phi_prime_denom[i, j, p]

    return phi_prime, phi_prime_denom, phi

In [2]:
def _phi_prime(X, Y, S11, S12, S22, S11_x, S11_y, S12_x, S12_y, S22_x, S22_y):
    
    '''
    Computes the linear interpolant of phi_prime and phi_prime_denom (denominator of phi prime) 
    based on the elements of the rate of strain tensor
    
    Parameters:
        X:                           array(Ny, Nx), X-meshgrid
        Y:                           array(Ny, Nx), Y-meshgrid
        Sij:                         array(Ny, Nx), ij-element of the rate of strain tensor over the gridded domain 
        Sij_x:                       array(Ny, Nx), derivative with respect to x-th coordinate of the ij-element of the rate of strain tensor over the gridded domain 
        Sij_y:                       array(Ny, Nx), derivative with respect to y-th coordinate of the ij-element of the rate of strain tensor over the gridded domain 
    
    Returns:
        Interpolant_phi_prime:       Interpolant-object for phi_prime over gridded domain and angles.
        Interpolant_phi_prime_denom: Interpolant-object for phi_prime_denom over gridded domain and angles.
    '''
    
    # Import interpolator from scipy
    from scipy.interpolate import RegularGridInterpolator
    
    # phi_prime and phi_prime_denom (=DOE) are arrays three dimensional arrays
    phi_prime, phi_prime_denom, phi = __phi_prime(S11, S12, S22, S11_x, S11_y, S12_x, S12_y, S22_x, S22_y)
    
    # Interpolant objects
    Interpolant_phi_prime = RegularGridInterpolator((Y[:,0], X[0,:], phi), phi_prime, method = "linear", bounds_error = False, fill_value = 0)
    Interpolant_phi_prime_denom = RegularGridInterpolator((Y[:,0], X[0,:], phi), phi_prime_denom, method = "linear")
    
    # return interpolant for phi_prime and for the denominator of phi_prime (=DOE)
    return Interpolant_phi_prime, Interpolant_phi_prime_denom